In [1]:
import json

import numpy as np
import pandas as pd

In [2]:
with open("hello_credit_config.json", "r") as f:
    file = json.load(f)

credit_mapper = file["credit_rating_mappings"]
credit_mapper_score = file['credit_rating_scale']

In [3]:
ratings = pd.read_excel("jalsh_ratings.xlsx", index_col=0)
dataset = pd.read_excel("dataset/jalsh_dataset.xlsx", sheet_name="data", header=[0, 1], index_col=0, parse_dates=True)

In [10]:
tickers = ratings['RATINGS_SCORES'].dropna()
data = dataset[tickers.index]

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
model = LinearRegression()

In [78]:
X = data.mean().unstack()

In [79]:
x = X.dropna()

In [61]:
X = X.T.dropna().T

In [27]:
model.fit(X, tickers)

LinearRegression()

In [29]:
from scipy.optimize import nnls

In [62]:
weights, residuals = nnls(X, tickers)

In [63]:
residuals

4.8773843239495775

In [36]:
X.columns

Index(['ASSET_TO_EQY', 'BS_CASH_NEAR_CASH_ITEM', 'BS_TOT_ASSET',
       'CASH_TO_TOT_ASSET', 'RETURN_ON_ASSET', 'SALES_GROWTH',
       'SALES_TO_TOT_ASSET', 'TANGIBLE_ASSETS', 'TOTAL_EQUITY',
       'TOT_DEBT_TO_TOT_ASSET', 'TOT_DEBT_TO_TOT_EQY'],
      dtype='object', name='Dates')

In [40]:
pd.DataFrame(weights, X.columns).replace(0, np.nan).dropna()

,0
Dates,
ASSET_TO_EQY,0.380224
CASH_TO_TOT_ASSET,0.137702
SALES_GROWTH,0.048581
SALES_TO_TOT_ASSET,1.011622
TOTAL_EQUITY,0.000012
TOT_DEBT_TO_TOT_ASSET,0.088819


In [81]:
y = ratings['RATINGS_SCORES'].loc[x.index]

In [82]:
weights, residuals = nnls(x, y)

In [83]:
w = pd.DataFrame(weights, x.columns).replace(0, np.nan).dropna()

In [86]:
w /= w.sum()
w

,0
Dates,
CASH_TO_TOT_ASSET,0.048915
CFO_TO_TOT_DEBT,0.597960
SALES_TO_TOT_ASSET,0.051109
TOT_DEBT_TO_EBITDA,0.277118
TOT_DEBT_TO_TOT_ASSET,0.024898


In [122]:
(weights @ x.T )

GLN SJ Equity    5.598095
BHG SJ Equity    4.702518
SAP SJ Equity    6.513242
ANG SJ Equity    5.256092
MNP SJ Equity    4.232223
AGL SJ Equity    5.500250
BTI SJ Equity    5.646322
dtype: float64

In [119]:
y

GLN SJ Equity    5.5
BHG SJ Equity    4.5
SAP SJ Equity    6.5
ANG SJ Equity    5.5
MNP SJ Equity    4.5
AGL SJ Equity    5.5
BTI SJ Equity    5.5
Name: RATINGS_SCORES, dtype: float64

In [89]:
model.fit(x, y)

LinearRegression()

In [91]:
model.intercept_

5.5900707234338745

In [95]:
from sklearn.linear_model import Lasso

In [96]:
m = Lasso(positive=True).fit(x, y)

In [99]:
pd.DataFrame(m.coef_, x.columns).replace(0, np.nan).dropna()

,0
Dates,
BS_TOTAL_AVAIL_LINE_OF_CREDIT,0.000022
TOT_DEBT_TO_TOT_EQY,0.013323


In [103]:
x['BS_TOTAL_LIABILITIES'] / x["BS_TOTAL_AVAIL_LINE_OF_CREDIT"]

GLN SJ Equity     9.513384
BHG SJ Equity     8.044649
SAP SJ Equity     6.871535
ANG SJ Equity     3.687600
MNP SJ Equity     8.784501
AGL SJ Equity     3.642070
BTI SJ Equity    13.830517
dtype: float64

In [110]:
x['TOT_DEBT_TO_EBITDA']

GLN SJ Equity    4.338971
BHG SJ Equity    1.230862
SAP SJ Equity    3.760033
ANG SJ Equity    2.662658
MNP SJ Equity    1.852745
AGL SJ Equity    3.023350
BTI SJ Equity    2.871179
Name: TOT_DEBT_TO_EBITDA, dtype: float64

In [127]:
X @ weights

S32 SJ Equity         NaN
BVT SJ Equity         NaN
ANH SJ Equity         NaN
PRX SJ Equity         NaN
GRT SJ Equity         NaN
SNT SJ Equity         NaN
NRP SJ Equity         NaN
GLN SJ Equity    5.598095
MTN SJ Equity         NaN
BHG SJ Equity    4.702518
RDF SJ Equity         NaN
HMN SJ Equity         NaN
MSP SJ Equity         NaN
CFR SJ Equity         NaN
FFB SJ Equity         NaN
SAP SJ Equity    6.513242
SOL SJ Equity         NaN
ANG SJ Equity    5.256092
FSR SJ Equity         NaN
PPH SJ Equity         NaN
MNP SJ Equity    4.232223
TKG SJ Equity         NaN
DSY SJ Equity         NaN
BAW SJ Equity         NaN
SSW SJ Equity         NaN
AGL SJ Equity    5.500250
BTI SJ Equity    5.646322
GFI SJ Equity         NaN
dtype: float64

In [131]:
ls = pd.DataFrame(weights, x.columns).replace(0, np.nan).dropna().index

In [137]:
w = weights[weights > 0]

In [141]:
yhats = (X[ls] @ w).dropna()

In [146]:
y_true = ratings['RATINGS_SCORES'].loc[yhats.index]

In [161]:
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_percentage_error

In [162]:
mean_absolute_percentage_error(yhats.astype(int), y_true.astype(int))
mean_absolute_percentage_error(yhats.astype(int), y_true.astype(int))

0.20526315789473684

In [158]:
pd.DataFrame((yhats.astype(int), y_true.astype(int))).T

,Unnamed 0,RATINGS_SCORES
S32 SJ Equity,5,5
BVT SJ Equity,5,6
ANH SJ Equity,5,4
GLN SJ Equity,5,5
MTN SJ Equity,5,6
BHG SJ Equity,4,4
MSP SJ Equity,10,6
CFR SJ Equity,5,4
SAP SJ Equity,6,6
SOL SJ Equity,5,6


In [193]:
X_true = X.T.dropna().T

In [204]:
X_true = X_true.loc[y_true.index]

In [206]:
X_true = X.loc[y_true.index][ls]

In [207]:
weights, residuals = nnls(X_true, y_true)
weights, residuals

(array([0.07759925, 1.85245425, 1.17025574, 0.28693158, 0.09101114]),
 3.911809260048702)

In [208]:
mean_absolute_percentage_error((X_true @ weights).astype(int), y_true.astype(int))

0.15137844611528822

In [188]:
(X_true @ weights)

S32 SJ Equity    5.374510
BVT SJ Equity    7.070677
ANH SJ Equity    5.554989
GLN SJ Equity    6.469017
MTN SJ Equity    6.087621
BHG SJ Equity    5.368090
MSP SJ Equity    6.445987
CFR SJ Equity    5.533802
SAP SJ Equity    6.792768
SOL SJ Equity    5.834166
ANG SJ Equity    5.292459
PPH SJ Equity    4.267847
MNP SJ Equity    5.125518
TKG SJ Equity    4.891411
BAW SJ Equity    5.610825
SSW SJ Equity    5.541101
AGL SJ Equity    5.189510
BTI SJ Equity    5.811893
GFI SJ Equity    6.190126
dtype: float64

In [217]:
X = pd.read_excel("dataset/features.xlsx", index_col=0, parse_dates=True)
y = pd.read_excel("dataset/target.xlsx", index_col=0, parse_dates=True)

C:\Users\103763\AppData\Local\Temp\ipykernel_8148\1757642365.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X = pd.read_excel("dataset/features.xlsx", index_col=0, parse_dates=True)
C:\Users\103763\AppData\Local\Temp\ipykernel_8148\1757642365.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  y = pd.read_excel("dataset/target.xlsx", index_col=0, parse_dates=True)


In [224]:
X = X.fillna(0)

In [227]:
nnls(X, y["numeric_rating"])

(array([1.35048988e+00, 0.00000000e+00, 0.00000000e+00, 1.70122779e-06,
        0.00000000e+00, 6.17702980e-01, 0.00000000e+00, 1.46626134e-05,
        1.21314529e-02, 4.25540160e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.46984922e-02, 1.24636024e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 7.49469476e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 8.08050975e-02, 7.42181811e-02,
        0.00000000e+00, 0.00000000e+00]),
 16.23417756291677)

In [226]:
y

,credit_rating,rating,numeric_rating
AGL SJ Equity,Baa2,Baa,5.5
ANG SJ Equity,Baa3,Baa,5.5
ANH SJ Equity,A3,A,4.5
BAW SJ Equity,Ba2,Ba,6.5
BHG SJ Equity,A1,A,4.5
...,...,...,...
UGPA3 BS Equity,Ba1,Ba,6.5
USIM5 BS Equity,Ba2,Ba,6.5
VALE3 BS Equity,Baa3,Baa,5.5
VAMO3 BS Equity,BB-,Ba,6.5
